In [ ]:
import os
ss = []
for i in os.walk("../"):
    for j in i[2]:
        if ".ipynb" in j and "-chechpoint" not in j:
            path = i[0] + "/" + j
            if "table_of_content" not in path:
                ss.append(path)

In [ ]:
for f in ss:
    ! jupyter nbconvert --to=html --template=github.tpl "$f"

In [63]:
link = '<li class="breadcrumb-item"><a href="{link}">{name}</a></li>'

entry = """
<div class="col-lg-4 ol-md-6 mb-2">
          <a href="{link}" class="text-dark">
          <div class="boxed p-2 rising">
            <div class="row align-items-center justify-content-between">
              <div class="col-10">
                <div class="media align-items-center">
                  <h4><i class="icon-folder mr-2"></i></h4>
                  <div class="media-body">
                    <h6 class="mb-0">{name}</h6>
                    <span class="text-muted">{type}</span>
                  </div>
                </div>
              </div>
            </div>
          </div>
        </a>
        </div>
""".replace("\n", "").replace(" ", "")

In [56]:
from bs4 import BeautifulSoup

html = open("../index2.html").read()

soup = BeautifulSoup(html)

path = soup.find(id="path")

In [61]:
BeautifulSoup(link, 'html.parser')

<li class="breadcrumb-item"><a href="{link}">{name}</a></li>